 # 基于Mindspore构造非对称损失函数---Focal Loss损失函数

 本小节主要介绍构造非对称损失函数的设计，使用MFocal Loss损失函数作为讲解实例。

Focal Loss是常见的非对称损失函数，该损失函数通过引入一个可调节的指数因子来降低容易被正确分类的样本的权重，从而使模型更加关注那些难以分类的样本。FocalLoss函数解决了类别不平衡的问题。

FocalLoss函数由Kaiming团队在论文 Focal Loss for Dense Object Detection 中提出，提高了图像目标检测的效果。

非对称损失函数是指在二元分类问题中，将错误分类的影响不对称地考虑在损失函数中。具体来
说，当将正样本错误分类为负样本时，损失函数的值与将负样本错误分类为正样本时所得到的损
失不同。

非对称损失函数可以提高模型对于特定类型错误分类的鲁棒性，常用于一些具有特定需求的应用场景，例如医疗诊断中对于假阴性或假阳性的不同考虑，或者金融风控中对于错过欺诈案件和误判为欺诈的不同风险评估。

### 函数如下：

$$FL(p_t)=-\alpha_t（1-p_t）^\upsilon log(p_t)$$

### 参数：

gamma(float)-gamma用于调整Focal Loss的权重曲线的陡峭程度。默认值：2.0。

weight(Union[Tensor,None])-Focal Loss的权重，维度为1。如果为None，则不使用权重。默认值：None。

reduction(str)-loss的计算方式。取值为”mean”，”sum”，或”none”。默认值：”mean”。

### 输入：

logits(Tensor) - shape为(N,C)、(N,C,H)、或(N,C,H,W)的Tensor，其中C是分类的数量，值大于1。如果shape为 (N,C,H,W)或 (N,C,H)，则H或H和W的乘积应与 labels的相同。

labels(Tensor)-shape为(N,C)、(N,C,H)、或(N,C,H,W)的Tensor，C的值为1，或者与logits的C相同。如果C不为1，则shape应与logits的shape相同，其中C是分类的数量。如果shape为 (N,C,H,W)或 (N,C,H) ，则H或H和W的乘积应与logits 相同。

### 输出：

Tensor或Scalar，如果reduction为”none”，其shape与logits相同。否则，将返回Scalar。

### 定义损失函数focal loss

In [15]:
import mindspore.nn as nn
import mindspore.ops as ops

class FocalLoss(nn.loss.SoftmaxCrossEntropyWithLogits):
    """
    Focal Loss for multi-class classification
    """
    def __init__(self, gamma=2.0, alpha=0.25, **kwargs):
        super(FocalLoss, self).__init__(**kwargs)
        self.gamma = gamma
        self.alpha = alpha
        self.sigmoid = nn.Sigmoid()  # sigmoid激活函数用于将logits转换为概率值
        self.reduce_sum = ops.ReduceSum() # 用于计算各维度上元素之和

    def construct(self, logits, label):
        """
        :param logits: model's predictions, shape of [batch_size, num_classes]
        :param label: ground truth labels, shape of [batch_size, num_classes]
        :return: focal loss
        """
        logits = self.sigmoid(logits)  # 将logits转换为概率值
        ce_loss = super(FocalLoss, self).construct(logits, label) # 计算交叉熵损失

        pt = self.reduce_sum(logits * label, axis=1)   # 计算类别预测概率pt
        fp = ops.Pow()(1 - pt, self.gamma)            # 计算focusing parameter fp
  
        weight = label * self.alpha + (1 - label) * (1 - self.alpha)  # 计算balanced weight

        fl_loss = weight * fp * ce_loss    # 计算focal loss
        fl_loss = self.reduce_sum(fl_loss, axis=1)  # 按照样本维度求和

        return fl_loss.mean()    # 返回平均focal loss

### 测试：

In [16]:
import mindspore as ms
import mindspore.nn as nn
import mindspore.ops as ops
import numpy as np

class TestFocalLoss:
    def test_focal_loss(self):
        batch_size = 4
        num_classes = 3
        gamma = 2.0
        alpha = 0.25

        # 创建 Focal Loss 对象
        focal_loss = nn.FocalLoss(gamma=gamma, alpha=alpha)

        # 生成随机的 logits 和 label
        logits = np.random.randn(batch_size, num_classes)
        label = np.random.randint(0, num_classes, (batch_size, num_classes))

        # 计算损失
        output = focal_loss(ms.Tensor(logits), ms.Tensor(label))
        assert output.shape == (), f"Focal Loss shape {output.shape} doesn't match expected shape ()"

        # 检查损失是否为标量
        assert output.asnumpy().dtype == np.float32, f"Focal Loss dtype {output.asnumpy().dtype} doesn't match expected dtype np.float32"